In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn')  
mpl.rcParams['font.family'] = 'serif'  
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

os.chdir(r"Z:\University\Course\FIN3380 Introduction to Financial Data Analysis\Lecture Materials\Lecture 4\lec4")

In [2]:
def isid(data,variables):      
    """Check and print whether the varaible or the combination of variables uniquely identify the dataset

    Args:
        data (Dataframe): the dataset
        variables (Array): the variables
    """

    dup = data.duplicated(variables, keep=False)
    if True in dup.values:
        print(str(variables)+" Do NOT uniquely identify this dataset") 
    else:
        print(str(variables)+" uniquely identify this dataset")

# Step 1. Load `crsp_beta.csv`, which contains 10 stocks returns. Then restrict the data from 2008-01-01 to 2010-12-31

In [3]:
crsp_beta = pd.read_csv("crsp_beta.csv")
crsp_beta['date'] = pd.to_datetime(crsp_beta['date'], format = '%m/%d/%Y')   #change the date 
crsp_beta

,permno,date,ret
0,11600,2000-01-31,0.047619
1,11600,2000-02-29,0.035354
2,11600,2000-03-31,0.109756
3,11600,2000-04-28,0.024229
4,11600,2000-05-31,0.055914
...,...,...,...
2117,91421,2018-08-31,-0.030256
2118,91421,2018-09-28,-0.073506
2119,91421,2018-10-31,-0.115297
2120,91421,2018-11-30,-0.019677


In [4]:
crsp_beta = crsp_beta.loc[(crsp_beta['date'] >= '2008-01-01') & (crsp_beta['date'] <= '2010-12-31')] 
crsp_beta

,permno,date,ret
96,11600,2008-01-31,-0.082408
97,11600,2008-02-29,-0.054950
98,11600,2008-03-31,-0.010118
99,11600,2008-04-30,0.006995
100,11600,2008-05-30,0.042964
...,...,...,...
2021,91421,2010-08-31,0.047066
2022,91421,2010-09-30,0.159950
2023,91421,2010-10-29,-0.043062
2024,91421,2010-11-30,0.076577


In [5]:
crsp_beta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 96 to 2025
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   permno  360 non-null    int64         
 1   date    360 non-null    datetime64[ns]
 2   ret     360 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 11.2 KB


In [6]:
crsp_beta.permno.value_counts()

11600    36
13610    36
39731    36
79089    36
83148    36
85459    36
86091    36
89509    36
90029    36
91421    36
Name: permno, dtype: int64

10 stocks, each with 36 prices

In [7]:
isid(crsp_beta,['permno','date'])

['permno', 'date'] uniquely identify this dataset


# Step 2. Get the risky portfolio weights with the minimum variance from mean-variance efficient frontier
+ Prepare the data into a return matrix of timeseries `ret_mat` for later use
+ Get the min variance portfolio weights `wgt` (recall lecture 2)

In [8]:
ret_mat = crsp_beta.set_index(['date','permno']).unstack()
ret_mat

ret                                                    \
permno         11600     13610     39731     79089     83148     85459   
date                                                                     
2008-01-31 -0.082408  0.060010  0.011765 -0.190567 -0.304478  0.026238   
2008-02-29 -0.054950 -0.052221 -0.203488 -0.052972  0.006438 -0.085884   
2008-03-31 -0.010118  0.028096 -0.035036 -0.050963 -0.101279  0.075832   
2008-04-30  0.006995  0.020749 -0.077508  0.171906 -0.037960  0.152206   
2008-05-30  0.042964  0.125434 -0.118616 -0.113998 -0.012330  0.029036   
2008-06-30 -0.091145  0.163556 -0.153271 -0.083254 -0.192260 -0.146357   
2008-07-31  0.093750  0.135982 -0.066667 -0.078775  0.372488 -0.121079   
2008-08-29 -0.026335 -0.088433 -0.166667  0.117252  0.057432  0.081120   
2008-09-30 -0.040954 -0.279078 -0.005714 -0.056484 -0.127796 -0.017847   
2008-10-31 -0.190730 -0.063918 -0.579304 -0.357067  0.141636  0.016091   
2008-11-28 -0.141540 -0.087004 -0.311010 -0.162534 -0.257754 -0.013519   
2008-12-31  0.084739  0.103785 -0.420000  0.098684 -0.051873  0.082028   
2009-01-30 -0.047096 -0.222898  0.862069 -0.308982 -0.164134 -0.164456   
2009-02-27 -0.140840 -0.242705 -0.333333  0.093588 -0.134545 -0.100043   
2009-03-31  0.163495  0.366858 -0.027778  0.244057  0.170168  0.108023   
2009-04-30  0.065922 -0.117029  0.428571  0.284076  0.213645  0.165534   
2009-05-29  0.022362  0.076190 -0.250000  0.108135  0.002959 -0.044018   
2009-06-30  0.046480 -0.110030 -0.200000  0.077887  0.001475  0.030893   
2009-07-31  0.090939  0.159798  0.333333  0.093854 -0.069219  0.045638   
2009-08-31  0.011095  0.228426  0.137500 -0.036447 -0.033228  0.031726   
2009-09-30  0.025801  0.041816  1.549450  0.120961  0.047463  0.030750   
2009-10-30 -0.045744 -0.124427 -0.211207 -0.060457  0.021875 -0.045714   
2009-11-30  0.010922  0.111329 -0.387978  0.154134 -0.039755  0.011613   
2009-12-31  0.057023  0.044723  0.357143  0.087844 -0.006369  0.057937   
2010-01-29 -0.046631 -0.058219  0.328947 -0.027116  0.017628 -0.035757   
2010-02-26 -0.013123  0.073333 -0.009901  0.110260 -0.045669 -0.058273   
2010-03-31  0.055606  0.120503  0.175000  0.041103  0.094059  0.051941   
2010-04-30  0.050745  0.070336  0.561702  0.032326 -0.075415  0.080036   
2010-05-28 -0.115143 -0.077619 -0.209809 -0.037474  0.009788 -0.036638   
2010-06-30 -0.076164 -0.056338 -0.165517 -0.074667 -0.048465 -0.037859   
2010-07-30  0.003678  0.122167  0.024793  0.141210  0.095076  0.171398   
2010-08-31 -0.074617 -0.107882 -0.274194  0.199243 -0.147287 -0.003221   
2010-09-30  0.152628  0.125628  0.055556  0.132659  0.129091  0.079705   
2010-10-29 -0.018142 -0.008433  0.005263  0.096672  0.012882  0.008009   
2010-11-30 -0.014973 -0.076038 -0.073298  0.146805 -0.004769  0.045829   
2010-12-31  0.106727  0.123153  0.000000  0.041833 -0.044728  0.091846   

                                                    
permno         86091     89509     90029     91421  
date                                                
2008-01-31  0.013197 -0.040442  0.109589  0.083019  
2008-02-29  0.001930  0.292820  0.097002  0.006969  
2008-03-31  0.164661  0.038710 -0.684887 -0.102076  
2008-04-30  0.052617  0.138107 -0.005102  0.020231  
2008-05-30 -0.087443  0.094703 -0.169231 -0.034939  
2008-06-30 -0.157606  0.022454 -0.098765 -0.037182  
2008-07-31  0.075380 -0.252045  0.116438 -0.186992  
2008-08-29  0.104892  0.066974  0.006135 -0.017500  
2008-09-30  0.158621 -0.119373 -0.134146 -0.165394  
2008-10-31 -0.044048 -0.172766 -0.788732 -0.396341  
2008-11-28 -0.068493 -0.297331 -0.200333 -0.040404  
2008-12-31  0.056180 -0.056045  2.501459 -0.210526  
2009-01-30 -0.190638 -0.072442 -0.250000 -0.133333  
2009-02-27 -0.084122 -0.206522 -0.079365 -0.046154  
2009-03-31  0.092422 -0.064631  0.103448  0.028226  
2009-04-30  0.183333  0.463547  0.734375 -0.129412  
2009-05-29  0.082629  0.214870  0.288288  0.900901  
2009-06-30 -0.002181 -0.131208 -0.097902 -0.099526  
2009-07-31 -0.010

The expected return of portfolio return $E(r_p)$:
$$ \large
E(r_p) = \underset{1 \times n}{w^T} \underset{n \times 1}{
    E(r_i)
    }  \\
w: \text{weights of each security in the portfolio}  \\
E(r_i): \text{expected return of each individual security}
$$

The volatility (risk) of portfolio return $\sigma^2(r_p)$:
$$ \large
\sigma^2(r_p) = \underset{1 \times n}{w^T} \underset{n \times n}{V} \underset{n \times 1}{w}    \\
w: \text{weights of each security in the portfolio}  \\
V: \text{covariance matrix of each indivicual security return}
$$

Sharpe ratio:
$$ \large
\text{Sharpe ratio} = \frac{E(r_p)}{\sigma^2(r_p)}
$$

Optmization problem:
$$ \large
\underset{w}{min} \text{ portfolio return volatility} =  \sigma^2(r_p) = \underset{1 \times n}{w^T} \  \underset{n \times n}{V} \ \underset{n \times 1}{w}  \text{such that}    \\
0 < w_i < 1 \text{ for each i} \\
\sum_{i=1}^n w_i = 1 
$$

In [27]:
import scipy.optimize as sco

def port_ret(weights):
    """The expected return based on the weights
    """
    return np.sum(ret_mat.mean() * weights)
    
def port_vol(weights):   
    """The risk based on the weights
    """
    return np.sqrt(np.dot(weights.T, np.dot(ret_mat.cov(), weights)))

#! Not used??? 
def min_func_sharpe(weights):
    """Sharpe ratio
    """
    return - port_ret(weights) / port_vol(weights)  

cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1}) # the sum equals one constraint 
noa = len(ret_mat.columns)  # the number of stocks
bnds = tuple((0, 1) for x in range(noa))    # the weight has to be between 0 and 1
eweights = np.array(noa * [1. / noa,])    # the initial equal weight

### Optimization algorithm to find the optimal weight for maximal sharpe ratio
opts = sco.minimize(port_vol, eweights,
                    method = 'SLSQP', bounds = bnds,
                    constraints = cons, tol = 1e-12)  

# The computed weights
opts.x

array([2.66749048e-01, 3.35700140e-02, 0.00000000e+00, 1.40880637e-17,
       5.36150579e-02, 3.22419190e-01, 2.83486173e-01, 9.55498484e-03,
       4.27357768e-17, 3.06055327e-02])

Need to use `droplevel()` function because the columns of `ret_mat` is a Multiindex. You can check this by calling `ret_mat.columns`

In [10]:
wgt = pd.DataFrame({'permno': ret_mat.columns.droplevel(), 'weight': (opts.x).round(6)}).set_index('permno')
wgt

,weight
permno,
11600,0.266749
13610,0.033570
39731,0.000000
79089,0.000000
83148,0.053615
85459,0.322419
86091,0.283486
89509,0.009555
90029,0.000000


# Step 3. Get the min variance portfolio monthly returns 
+ Based on the weight, we want to find the monthly return: We can use `.dot` to compute the monthly returns  (`date` is still the index, so we use `.reset_index()` to get a column for `date`)

In [11]:
# compute portfolio returns, and convert the array into a dataframe with crsp_wide.index (i.e. date) as index, and ret_pf as column name
ret_pf = pd.DataFrame(data=np.dot(ret_mat, wgt), index=ret_mat.index, columns=['ret_pf']) 
ret_pf = ret_pf.reset_index() ### get date out of the row index
ret_pf.rename(columns={"ret_pf": "Rit"}, inplace=True)
ret_pf

,date,Rit
0,2008-01-31,-0.021937
1,2008-02-29,-0.040198
2,2008-03-31,0.061189
3,2008-04-30,0.066456
4,2008-05-30,-0.000581
5,2008-06-30,-0.121921
6,2008-07-31,0.023743
7,2008-08-29,0.049080
8,2008-09-30,0.005865
9,2008-10-31,-0.066509


Transform the `date` column of this dataframe into `YearMonth` type, so that it will be compatable with the risk-free rate dataframe format later.

In [12]:
ret_pf['date'] = ret_pf['date'].dt.strftime("%Y%m")

In [13]:
ret_pf

,date,Rit
0,200801,-0.021937
1,200802,-0.040198
2,200803,0.061189
3,200804,0.066456
4,200805,-0.000581
5,200806,-0.121921
6,200807,0.023743
7,200808,0.049080
8,200809,0.005865
9,200810,-0.066509


# Step 4. Load the `ff.csv` data and use market return as benchmark
+ Load the data with only columns `Month`, `RF`, and `Mkt-RF`.   
  The `RF` column are the $R_{ft}$   
  The `Mkt-RF` column are the $R_{mt} - R_{ft}$
+ Set the `Month` to be `datetime` format, which  is the same as `prot_ret`  (note we are using different names for both DataFrames).    
  Also note that the date are different (one is the first day for each month, and the other is the last day of the month), so we keep only the month and year to make them the same.

In [14]:
ff = pd.read_csv('ff.csv', usecols = ['Month', 'RF', 'Mkt-RF'])
ff['Month'] = pd.to_datetime(ff['Month']).dt.strftime("%Y%m") 
ff.rename(columns={"Mkt-RF": "Rmt-Rft", "RF": "Rft"}, inplace=True)
ff

,Rmt-Rft,Rft,Month
0,-0.0394,0.0043,196812
1,-0.0125,0.0053,196901
2,-0.0584,0.0046,196902
3,0.0264,0.0046,196903
4,0.0146,0.0053,196904
...,...,...,...
596,0.0344,0.0016,201808
597,0.0006,0.0015,201809
598,-0.0768,0.0019,201810
599,0.0169,0.0018,201811


In [15]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rmt-Rft  601 non-null    float64
 1   Rft      601 non-null    float64
 2   Month    601 non-null    object 
dtypes: float64(2), object(1)
memory usage: 14.2+ KB


# Step 5. Combine `ff` and `port_ret` and calculate the excess return of the portfolio
+ We use `merge` to combine both DataFrames, and we choose the `data` column from `port_ret` to match with the `Month` column from `ff`.
+ Then we creat a new column `port-rf` from the difference between the column `ret` and the column `RF`

In [16]:
ret_pf_new = pd.merge(ret_pf, ff, left_on='date', right_on='Month', how='left').drop('Month', axis = 1)  
ret_pf_new

,date,Rit,Rmt-Rft,Rft
0,200801,-0.021937,-0.0636,0.0021
1,200802,-0.040198,-0.0309,0.0013
2,200803,0.061189,-0.0093,0.0017
3,200804,0.066456,0.0460,0.0018
4,200805,-0.000581,0.0186,0.0018
5,200806,-0.121921,-0.0844,0.0017
6,200807,0.023743,-0.0077,0.0015
7,200808,0.049080,0.0153,0.0013
8,200809,0.005865,-0.0924,0.0015
9,200810,-0.066509,-0.1723,0.0008


In [17]:
# This is the dataset for regression
ret_pf_new['Rit-Rft'] = ret_pf_new["Rit"] - ret_pf_new["Rft"]
ret_pf_new

,date,Rit,Rmt-Rft,Rft,Rit-Rft
0,200801,-0.021937,-0.0636,0.0021,-0.024037
1,200802,-0.040198,-0.0309,0.0013,-0.041498
2,200803,0.061189,-0.0093,0.0017,0.059489
3,200804,0.066456,0.0460,0.0018,0.064656
4,200805,-0.000581,0.0186,0.0018,-0.002381
5,200806,-0.121921,-0.0844,0.0017,-0.123621
6,200807,0.023743,-0.0077,0.0015,0.022243
7,200808,0.049080,0.0153,0.0013,0.047780
8,200809,0.005865,-0.0924,0.0015,0.004365
9,200810,-0.066509,-0.1723,0.0008,-0.067309


# Step 6: Run regression

Regression formula:
$$ \large
R_{it} - R_{ft} = \alpha_i + \beta_i (R_{mt} - R_{ft}) + \epsilon_{it}  \\
\begin{aligned}
& R_{it}: \text{return of min variance portfolio over time}   \\
& R_{ft}: \text{risk-free rate over time} \\
& R_{it} - R_{ft}: \text{excess return of the min variance portfolio over time} \\
& \alpha_i: \text{extra return / abnormal return of the min variance portfolio}   \\
& \beta_i: \text{the correlation between the min variance portfolio and market portfolio} \\
& R_{mt}: \text{return of market portfolio over time} \\
& R_{mt} - R_{ft}: \text{excess return of the market}  \\
& \epsilon_{it}: \text{noise / error}   
& \end{aligned}
$$

If you need to install statsmodel first, `pip install statsmodels`   
For more on OLS, see [this website](https://www.statsmodels.org/devel/examples/notebooks/generated/ols.html)   
For more on `statsmodels RegressionResults`, see [this webiste](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html)
For more on t-test, see [this website](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.RegressionResults.t_test.html#statsmodels.regression.linear_model.RegressionResults.t_test)   
For more functions of `statsmodels` module, see [this website](https://pypi.org/project/statsmodels/)

In [18]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

X = ret_pf_new["Rmt-Rft"]         # Set X values, explanatory/independent rhs variables
X = sm.add_constant(X) # add a column of 1 to X matrix
y = ret_pf_new["Rit-Rft"]       # Set y values, dependent/lhs variable

model = sm.OLS(y, X)    # generate the model using OLS model with input y vector and X matrix
res = model.fit()      # Run model to get the results
res.summary()          # Get the summary of the results 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Rit-Rft   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.682
Method:                 Least Squares   F-statistic:                     76.12
Date:                Tue, 14 Jun 2022   Prob (F-statistic):           3.40e-10
Time:                        17:29:58   Log-Likelihood:                 68.354
No. Observations:                  36   AIC:                            -132.7
Df Residuals:                      34   BIC:                            -129.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0068      0.006      1.092      0.283      -0.006       0.019
Rmt-Rft        0.8422      0.097      8.725      0.000       0.646       1.038
==============================================================================
Omnibus:                        0.158   Durbin-Watson:                   1.748
Prob(Omnibus):                  0.924   Jarque-Bera (JB):                0.362
Skew:                           0.080   Prob(JB):                        0.834
Kurtosis:                       2.535   Cond. No.                         15.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
print(res.params)

alpha = res.params[0]
beta = res.params[1]
print('alpha = ', alpha, 'beta = ', beta)

const      0.006786
Rmt-Rft    0.842240
dtype: float64
alpha =  0.006786360718114504 beta =  0.8422398718499823


In [20]:
# The standard errors of the parameter estimates.
res.bse 

const      0.006215
Rmt-Rft    0.096536
dtype: float64

In [21]:
res.pvalues  ### P values, null is const=0, Rmt-Rft=0 respectively

const      2.825168e-01
Rmt-Rft    3.399570e-10
dtype: float64

t-test to test whether the excess return of min variance portfolio $\alpha_i$ is significant from 0

In [22]:
h1 = 'const = 0'
res.t_test(h1)   

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0068      0.006      1.092      0.283      -0.006       0.019

Suppose confidence level $\alpha = 0.05, t_{n-1, \alpha / 2} = t_{35, 0.025} = 2.030108$, the critical region is $(-\infty, -2.030108) \cup (2.03108, +\infty)$   
The t statistic `1.092` is NOT in critical region, so we CAN'T reject the null hypothesis that `const = 0`.   
**In conclusion, the execss return of min variance portfolio $\alpha_i$ is NOT significant from 0 (insignificant).**

t-test to test whether the min variance portfolio and market portfolio are perfectly correlated (correlation $\beta_i = 1$). 

In [28]:
h2 = 'Rmt-Rft = 1'  
res.t_test(h2)    

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.8422      0.097     -1.634      0.111       0.646       1.038

Suppose confidence level $\alpha = 0.05, t_{n-1, \alpha / 2} = t_{35, 0.025} = 2.030108$, the critical region is $(-\infty, -2.030108) \cup (2.03108, +\infty)$   
The t statistic `-1.634` is NOT in critical region, so we CAN'T reject the null hypothesis that $\beta_i = 1$.   
**In conclusion, the correlation between min variance portfolio and market portfolio $\beta_i$ is NOT significant from 1 (insignificant).**

# Conclusion

The execss return of min variance portfolio $\alpha_i$ is NOT significant from 0 (insignificant).   
This agrees with the CAPM theory.

The correlation between min variance portfolio and market portfolio $\beta_i$ is NOT significant from 0 (insignificant).   
This agrees with the CAPM theory

This dataset verifies the CAPM theory